In [15]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import cm


from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

from scipy.ndimage import gaussian_filter1d

#from itertools import product
from pynwb import NWBHDF5IO
from nlb_tools.nwb_interface import NWBDataset

import warnings
warnings.filterwarnings("ignore")

In [40]:
import numpy as np
import pandas as pd
import h5py

import torch

from nlb_tools.nwb_interface import NWBDataset

In [42]:
io = NWBHDF5IO(
    file=h5py.File('data/000070/sub-Jenkins/sub-Jenkins_ses-20090912_behavior+ecephys.nwb'),
    mode='r',
    load_namespaces=True
)

In [44]:
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x5359756368\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5293908240\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0


In [47]:
trialtable = nwb.trials.to_dataframe()
neurons = nwb.units.to_dataframe()['spike_times']

In [ ]:

trialtable = trialtable[trialtable['correct_reach'] == 0].reset_index()
neurons = nwb.units.to_dataframe()['spike_times'] # Neuron id x Spike Time
time = np.round(nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].timestamps[:]+ 0.0001,decimals=3)
hand = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].data[:] # x, y 
cursor = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Cursor'].data[:] # x, y
gocue = np.round(trialtable['go_cue_time'], decimals=3)
startmov = np.round(trialtable['move_begins_time'], decimals=3)
endmov = np.round(trialtable['move_ends_time'], decimals=3)
reactiontime = np.round(trialtable['reaction_time'], decimals=3)
target_pos = trialtable['target_positions']
num_neurons = neurons.shape[0]

targeton = np.round(trialtable['target_presentation_time'], decimals=3)